Ejercicio: Uso de un modelo entrenado con nuevos datos
En la Unidad 3, creamos un modelo básico que nos permitía encontrar la relación entre el tamaño del arnés de un perro y el tamaño de sus botas. Mostramos cómo este modelo se podía utilizar para hacer una predicción sobre un perro nuevo, no visto previamente.

Es habitual construir, entrenar y luego utilizar un modelo mientras estamos aprendiendo sobre aprendizaje automático; pero en el mundo real, no queremos entrenar el modelo cada vez que queremos hacer una predicción.

Consideremos nuestro escenario de la tienda de equipos para avalanchas y perros:

Queremos entrenar el modelo una sola vez y luego cargarlo en el servidor que gestiona nuestra tienda online.
Aunque el modelo se ha entrenado en un conjunto de datos que hemos descargado de Internet, en realidad queremos utilizarlo para estimar el tamaño de las botas de los perros de nuestros clientes que no están en este conjunto de datos.
¿Cómo podemos hacerlo?

Vamos a

Crearemos un modelo básico
Guardarlo en el disco
Cargarlo desde el disco
Utilizarlo para hacer predicciones sobre un perro que no estaba en el conjunto de datos de entrenamiento.
Cargar el conjunto de datos
Empecemos abriendo el conjunto de datos desde el archivo.

In [ ]:
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-boot-harness.csv

# Load a file containing dog's boot and harness sizes
data = pandas.read_csv('doggy-boot-harness.csv')

# Print the first few rows
data.head()

Crear y entrenar un modelo
Como hemos hecho antes, crearemos un modelo de regresión lineal simple y lo entrenaremos en nuestro conjunto de datos.

In [ ]:
import statsmodels.formula.api as smf

# Fit a simple model that finds a linear relationship
# between boot size and harness size, which we can use later
# to predict a dog's boot size, given their harness size
model = smf.ols(formula = "boot_size ~ harness_size", data = data).fit()

print("Model trained!")

Guardar y cargar un modelo
Nuestro modelo está listo para ser utilizado, pero aún no lo necesitamos. Guardémoslo en el disco.

In [ ]:
import joblib

model_filename = './avalanche_dog_boot_model.pkl'
joblib.dump(model, model_filename)

print("Model saved!")

In [ ]:
model_loaded = joblib.load(model_filename)

print("We have loaded a model with the following parameters:")
print(model_loaded.params)

Ponerlo todo junto
En nuestro sitio web, vamos a querer tomar el arnés del perro de nuestro cliente, a continuación, calcular el tamaño de la bota de su perro utilizando el modelo que ya hemos entrenado.

Pongamos todo junto para hacer una función que cargue el modelo desde el disco, y lo utilice para predecir la altura de la bota de nuestro cliente.

In [ ]:
# Let's write a function that loads and uses our model
def load_model_and_predict(harness_size):
    '''
    This function loads a pretrained model. It uses the model
    with the customer's dog's harness size to predict the size of
    boots that will fit that dog.

    harness_size: The dog harness size, in cm 
    '''

    # Load the model from file and print basic information about it
    loaded_model = joblib.load(model_filename)

    print("We've loaded a model with the following parameters:")
    print(loaded_model.params)

    # Prepare data for the model
    inputs = {"harness_size":[harness_size]} 

    # Use the model to make a prediction
    predicted_boot_size = loaded_model.predict(inputs)[0]

    return predicted_boot_size

# Practice using our model
predicted_boot_size = load_model_and_predict(45)

print("Predicted dog boot size:", predicted_boot_size)

Uso en el mundo real,
Lo hemos conseguido; podemos predecir la talla de las botas de un perro de avalancha basándonos en la talla de su arnés. Nuestro último paso es utilizar esto para advertir a la gente si pueden estar comprando botas de perro de tamaño incorrecto.

Como ejemplo, haremos una función que acepte la talla del arnés, la talla de las botas seleccionadas y devuelva un mensaje para el cliente. Integraríamos esta función en nuestra tienda online.

In [ ]:
def check_size_of_boots(selected_harness_size, selected_boot_size):
    '''
    Calculates whether the customer has chosen a pair of doggy boots that 
    are a sensible size. This works by estimating the dog's actual boot 
    size from their harness size.

    This returns a message for the customer that should be shown before
    they complete their payment 

    selected_harness_size: The size of the harness the customer wants to buy
    selected_boot_size: The size of the doggy boots the customer wants to buy
    '''

    # Estimate the customer's dog's boot size
    estimated_boot_size = load_model_and_predict(selected_harness_size)

    # Round to the nearest whole number because we don't sell partial sizes
    estimated_boot_size = int(round(estimated_boot_size))

    # Check if the boot size selected is appropriate
    if selected_boot_size == estimated_boot_size:
        # The selected boots are probably OK
        return f"Great choice! We think these boots will fit your avalanche dog well."

    if selected_boot_size < estimated_boot_size:
        # Selected boots might be too small 
        return "The boots you have selected might be TOO SMALL for a dog as "\
               f"big as yours. We recommend a doggy boots size of {estimated_boot_size}."

    if selected_boot_size > estimated_boot_size:
        # Selected boots might be too big 
        return "The boots you have selected might be TOO BIG for a dog as "\
               f"small as yours. We recommend a doggy boots size of {estimated_boot_size}."
    

# Practice using our new warning system
check_size_of_boots(selected_harness_size=55, selected_boot_size=39)

Cambie selected_harness_size y selected_boot_size en el ejemplo anterior y vuelva a ejecutar la celda para ver esto en acción.

Resumen
Muy bien hecho. Hemos creado un sistema que puede predecir si los clientes están comprando botas para perros que pueden no ajustarse a su perro de avalancha, basándose únicamente en el tamaño del arnés que están comprando.

En este ejercicio, hemos practicado <br>

Creación de modelos básicos <br>
Entrenándolos y guardándolos en disco <br>
Cargarlos desde el disco <br>
Hacer predicciones con ellos utilizando nuevos conjuntos de datos